In [16]:
import numpy as np
import pandas as pd
import sys, os
import re 
import csv
import shlex
import json
import datetime

In [5]:
logfile = '/home/tech/haproxy_log_analysis/haproxy.log'

In [29]:
pattern_json = re.compile(r'{.*}')
pattern_select_words = re.compile(r'[^:,{}]+')

pattern_method = re.compile(r'(^[A-Z]+\b)')
pattern_url = re.compile(r'([/bh].*)\s')
pattern_protocol = re.compile(r'(\s[H].*)')

In [30]:
month = input("Enter the month : ")

Enter the month : 09


In [31]:
day = input("Enter the date : ")

Enter the date : 23


In [32]:
date = '2017-'+str(month)+'-'+str(day)

In [40]:
content = []
with open(logfile) as file:
    for line in file:
        for match in re.findall(pattern_json, line):
            string_json = match
        string_json = re.sub(pattern_select_words, r'"\g<0>"', string_json)
        try:
            json_data = json.loads(string_json)
        except ValueError:
            continue
        data = []
        timestamp = int(json_data.get('timestamp'))
        day = datetime.datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d')
        
        if day == date:
            i = 0
            data.append(day)
            data.append(datetime.datetime.fromtimestamp(timestamp).strftime('%H:%M:%S'))

            http_request = str(json_data.get('http_request',None))
            for url in re.findall(pattern_url, http_request):
                data.append(url)
            for method in re.findall(pattern_method, http_request):
                data.append(method)
            for protocol in re.findall(pattern_protocol, http_request):
                data.append(protocol)

            data.append(json_data.get('upstream_addr'))
            data.append(json_data.get('remote_addr'))
            data.append(int(json_data.get('http_status')))
            data.append(int(json_data.get('session_duration')))
            data.append(int(json_data.get('upstream_connect_time')))
            data.append(int(json_data.get('upstream_response_time')))
            
            content.append(data)
 
        i = i+1
        if(i == 100):
            break
            
    


{"type":"haproxy","timestamp":"1506186764","http_status":"200","http_request":"GET /supplier/marketManageList.html HTTP/1.1","remote_addr":"172.31.22.212","bytes_read":"170575","upstream_addr":"94.130.54.24","backend_name":"app_cluster","retries":"0","bytes_uploaded":"1377","upstream_response_time":"41","upstream_connect_time":"6","session_duration":"2030","termination_state":"--"}


In [42]:
columns = ['day', 'time', 'url', 'method', 'protocol', 'upstream_addr',
           'remote_addr', 'http_status', 'session_duration',
           'upstream_connect_time', 'upstream_response_time']

df = pd.DataFrame(content, columns=columns)
df.to_csv('/home/tech/haproxy_log_analysis/haproxy.csv', index = None)

In [60]:
new = df.groupby(['remote_addr','upstream_addr','method'])['upstream_response_time'].agg(['sum','count'])
n=new.reset_index()
n1=n.groupby(['remote_addr','upstream_addr','method']).sum()
n1.rename(columns={'count':'hits','sum':'UPST_RT_duration'},inplace=True)

KeyError: 'remote_addr'

In [44]:
a=n1.reset_index()
a['UPST_RT_min']=(a.UPST_RT_duration)/(60)
a['UPST_RT_TOTAL'] = pd.to_timedelta((60*a['UPST_RT_min']).astype('int'), unit='s')
a1 = a.sort_values(by=['UPST_RT_TOTAL'], ascending=False)
a1 = a1.head(100)
del a1['UPST_RT_duration']
del a1['UPST_RT_min']
a1['UPST_RT_avg']=(a1.UPST_RT_TOTAL)/(a1.hits)
a1 = a1.reset_index()
del a1['index']

In [59]:
l = []
l = a1.remote_addr
sd = []
p = []
for i in l:
    a = df[df.remote_addr == i]
    x = np.std(a['upstream_response_time'])
    sd.append(x)
    p1 = np.percentile(a['upstream_response_time'],90)
    p.append(p1)
sd_array = pd.Series(sd)
p_array = pd.Series(p)
a1['SD'] = sd_array.values
a1['90_percentile'] = p_array.values
a1.to_csv('/home/tech/haproxy_log_analysis/haproxy_log_report.csv', index = None)

AttributeError: 'DataFrame' object has no attribute 'remote_addr'

In [4]:
import glob, os
os.chdir("/home/tech/haproxy_log_analysis/test")
for logfiles in glob.glob("*.log"):
    with open(logfiles) as file:
        for line in file:
            print(line)

1

3

2

